In [1]:
#output_folder = '/home/jnaiman/data/morgan/' # HAL
output_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/' # local

#ender = '_regular'
#ender = '_small'
#ender = '_small_words'
ender = '_small_words_pageLevel'


vocabulary_file = "data/vocabulary_new_pages_new"+ender+".pkl"
train_file = "data/train_aligned_new_pages"+ender+".pkl"
val_file = "data/dev_aligned_new_pages"+ender+".pkl"

output_folder+"/data/train_aligned_new_pages"+ender+".pkl"

window_length = 100 # for subsetting the data

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.lm import Vocabulary
import torch
import torch.utils.data as tud
# import sys
# sys.path.append("../../lib")
#from metrics import levenshtein
import pickle
from pathlib import Path
import re
import gc
import time
import glob

Read in data:

In [3]:
#'/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan//data/vocabulary_new_pages_new_small_words.pkl'

In [4]:
#output_folder+vocabulary_file
output_folder+vocabulary_file

'/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/data/vocabulary_new_pages_new_small_words_pageLevel.pkl'

In [5]:
with open(output_folder+vocabulary_file, "rb") as file:
    vocabulary = pickle.load(file)

In [6]:
len(vocabulary)

408

In [7]:
output_folder+train_file

'/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/data/train_aligned_new_pages_small_words_pageLevel.pkl'

In [8]:
#'/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/data/train_aligned_new_pages_small_words.pkl'

In [9]:
train = pd.read_pickle(output_folder+train_file)
train.shape

(3041135, 2)

In [10]:
val = pd.read_pickle(output_folder+val_file)
val.shape

(766032, 2)

Char to integer/integer to char:

In [11]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
len(char2i)

411

In [12]:
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
len(i2char)

411

Padding sequences?

In [13]:
#length = 100
#window_length = 100 # I think this is the same as length?

output = []
for s in tqdm(train.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_source.shape)

output = []
for s in tqdm(train.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_target.shape)

  0%|          | 0/3041135 [00:00<?, ?it/s]

torch.Size([3041135, 102])


  0%|          | 0/3041135 [00:00<?, ?it/s]

torch.Size([3041135, 102])


Testing the padding maybe?

In [14]:
train.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_source[0].tolist()]))

True

In [15]:
train.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_target[0].tolist()]))

True

In [16]:
torch.save(train_source, output_folder+"data/train_source_new_pages"+ender+".pt")

In [17]:
torch.save(train_target, output_folder+"data/train_target_new_pages"+ender+".pt")

In [18]:
del train

Same thing with the dev set:

In [19]:
dev = pd.read_pickle(output_folder + val_file)
dev.shape

(766032, 2)

In [20]:
output = []
for s in tqdm(dev.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_source.shape)

output = []
for s in tqdm(dev.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_target.shape)

  0%|          | 0/766032 [00:00<?, ?it/s]

torch.Size([766032, 102])


  0%|          | 0/766032 [00:00<?, ?it/s]

torch.Size([766032, 102])


In [21]:
dev.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_source[0].tolist()]))

True

In [22]:
dev.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_target[0].tolist()]))

True

Save all the things for training later:

In [23]:
torch.save(dev_source, output_folder+"data/dev_source_new_pages"+ender+".pt")

In [24]:
torch.save(dev_target, output_folder+"data/dev_target_new_pages"+ender+".pt")

In [25]:
with open(output_folder+"data/char2i_new_pages"+ender+".pkl", "wb") as file:
    pickle.dump(char2i, file)

In [26]:
with open(output_folder+"data/i2char_new_pages"+ender+".pkl", "wb") as file:
    pickle.dump(i2char, file)

In [27]:
del dev